In [1]:
import cv2 
import numpy
import matplotlib.pyplot as plt
import scipy.ndimage

In [2]:
orig_gray = cv2.imread('GRAY.JPG')
orig_RGB_quarter = cv2.imread('RGB_quater.JPG')
orig_RGB_half = cv2.imread('RGB_half.JPG')

# Pan Sharpen Algorithm

In [17]:
def pan_sharpen(gray_img, color_img, unique_num):
    
    height_gray, width_gray, ch_gray = gray_img.shape 
    height_quarter, width_quarter, ch_quarter = color_img.shape 
    
    # up sample low resolution image
    up_sampled_img = scipy.ndimage.zoom(color_img, (height_gray/height_quarter, width_gray/width_quarter, 1))
    
    cv2.imwrite(f'up_sampled_colored_img_{unique_num}.jpg', up_sampled_img)
    
    # image aligning 
    orb_detector = cv2.ORB_create(5000)
    
    kp1, d1 = orb_detector.detectAndCompute(gray_img, None)
    kp2, d2 = orb_detector.detectAndCompute(up_sampled_img, None)
    
    matcher = cv2.BFMatcher(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    
    matches = matcher.match(d1, d2, None)
    
    matches.sort(key = lambda x: x.distance, reverse=False)
    
    matches = matches[:int(len(matches)*0.05)] 
    no_of_matches = len(matches)
    
    p1 = np.zeros((no_of_matches, 2), dtype=np.float32) 
    p2 = np.zeros((no_of_matches, 2), dtype=np.float32) 

    for i in range(len(matches)): 
        p1[i, :] = kp1[matches[i].queryIdx].pt 
        p2[i, :] = kp2[matches[i].trainIdx].pt
        
    # find homography
    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC) 
    
    height, width, channels = gray_img.shape
    inversed_homography = np.linalg.inv(homography)

    transformed_img = cv2.warpPerspective(up_sampled_img, 
                        inversed_homography, (width, height))
    
    cv2.imwrite(f'aligned_upsampling_image_{unique_num}.jpg', transformed_img)
    

    # convert to LUV color space
    transformed_img_LUV = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2LUV)
    gray_LUV = cv2.cvtColor(gray_img, cv2.COLOR_BGR2LUV)
    
    gray_LUV[:, :, 1] = transformed_img_LUV[:, :, 1]
    gray_LUV[:, :, 2] = transformed_img_LUV[:, :, 2]
    
    # convert from LUV to BGR
    output = cv2.cvtColor(gray_LUV, cv2.COLOR_LUV2BGR)
    
    cv2.imwrite(f'pan_sharped_image_{unique_num}.jpg', output)
    
    cv2.imshow('Original Image', gray_img)
    cv2.imshow('Pan Sharped Image', output)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
unique_num = 1

pan_sharpen(orig_gray, orig_RGB_half, unique_num-1)